In [1]:
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, BatchNormalization, Dropout, Embedding
from keras.layers import Activation, Concatenate 
from keras.initializers import RandomNormal
from matplotlib import pyplot
from keras.utils import to_categorical, Progbar
from keras.datasets import cifar10
import numpy as np

In [2]:
def define_discriminator(in_shape=(32,32,3), n_classes=10):

    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    # layer 1
    fe = Conv2D(16, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # layer 2
    fe = Conv2D(32, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # layer 3
    fe = Conv2D(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # layer 4
    fe = Conv2D(128, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # layer 5
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # layer 6
    fe = Conv2D(128, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)

    fe = Flatten()(fe)
    # real/fake output
    out1 = Dense(1, activation='sigmoid')(fe)
    # class label output
    out2 = Dense(n_classes, activation='softmax')(fe)
 
    model = Model(in_image, [out1, out2])

    opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)
    model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=opt)
    return model

In [3]:
def define_generator(latent_dim, n_classes=10):
    
    init = RandomNormal(stddev=0.02)
    # label input
    in_label = Input(shape=(n_classes,))
    in_lat = Input(shape=(latent_dim,))
    # layer 1
    merge = Concatenate()([in_lat, in_label])
    n_nodes = 384 
    gen = Dense(n_nodes, kernel_initializer=init)(merge)
    gen = Activation('relu')(gen)
    gen = Reshape((1, 1, 384))(gen)
    # layer 2
    gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization()(gen)
    gen = Activation('relu')(gen)
    # layer 3
    gen = Conv2DTranspose(96, (5,5), strides=(4,4), padding='same', kernel_initializer=init)(gen)
    gen = BatchNormalization()(gen)
    gen = Activation('relu')(gen)
    # layer 4
    gen = Conv2DTranspose(3, (5,5), strides=(4,4), padding='same', kernel_initializer=init)(gen)
    out_layer = Activation('tanh')(gen)

    model = Model([in_lat, in_label], out_layer)
    return model

In [15]:
def define_gan(g_model, d_model):
	d_model.trainable = False
	gan_output = d_model(g_model.output)
	model = Model(g_model.input, gan_output)
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=opt)
	return model

In [5]:
def load_real_samples():
    # load dataset
    (trainX, trainy), (testX, testy) = cifar10.load_data()

    X = np.concatenate((trainX, testX))
    Y = np.concatenate((trainy, testy))
    X = X.astype('float32')
 
    X = (X - 127.5) / 127.5
    Y = to_categorical(Y)
    print(X.shape, Y.shape)
    return [X, Y]

def generate_real_samples(dataset, n_samples):
	images, labels = dataset
	ix = randint(0, images.shape[0], n_samples)
	X, labels = images[ix], labels[ix]
	y = ones((n_samples, 1))
	return [X, labels], y

def denorm(x):
  out = (x + 1) / 2
  return out.clamp(0, 1)

def generate_latent_points(latent_dim, n_samples, n_classes=10, uniform=False):
    x_input = randn(latent_dim * n_samples)
    z_input = x_input.reshape(n_samples, latent_dim)
 
    if not uniform:
        labels = randint(0, n_classes, n_samples)
    else:
        labels = np.array([[j for j in range(int(n_samples/n_classes))] for i in range(n_classes)]).reshape(-1, 1)
    labels = to_categorical(labels, num_classes=n_classes)
    return [z_input, labels]

def generate_fake_samples(generator, latent_dim, n_samples, uniform=False):
	z_input, labels_input = generate_latent_points(latent_dim, n_samples, uniform=uniform)
	images = generator.predict([z_input, labels_input])
	y = zeros((n_samples, 1))
	return [images, labels_input], y

class_names = ['airplane','car','bird','cat','deer',
               'dog','frog','horse','ship','truck']
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    fig = pyplot.figure(figsize=(10,10))
    fig.suptitle(f'epoch {step}')
    [X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples, uniform=True)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    for i in range(100):
     
        pyplot.subplot(10, 10, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i, :, :, :])
        if i < 10:
            pyplot.title(class_names[i])
    
    pyplot.show()
    filename1 = 'image_generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    # filename2 = 'model_%04d.h5' % (step+1)
    # g_model.save(filename2)
    # print('>Saved: %s and %s' % (filename1, filename2))

In [6]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=2, n_batch=100, history=None):
   
    if history is None:
        history = {'dr1':[], 'dr2':[], 'df1':[], 'df2':[], 'g1':[], 'g2':[]}
    
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    
    n_steps = bat_per_epo
   
    half_batch = int(n_batch / 2)
    
    for e in range(n_epochs):
        
        print(f'epoch: {e+1}')
        progbar = Progbar(target=n_steps)
        for i in range(n_steps):
           
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
           
            _,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])

            [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

            _,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])

            [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)

            y_gan = ones((n_batch, 1))

            _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])

            progbar.update(i, [('dr1', d_r1), ('dr2', d_r2), ('df1', d_f), ('df2', d_f2), ('g1', g_1), ('g2', g_2)])
            history['dr1'].append(d_r1)
            history['dr2'].append(d_r2)
            history['df1'].append(d_f)
            history['df2'].append(d_f2)
            history['g1' ].append(g_1)
            history['g2' ].append(g_2)
            # print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2, d_f,d_f2, g_1,g_2))
            # evaluate the model performance every 'epoch'
        summarize_performance(e+1, g_model, latent_dim)
    return history

In [7]:
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
discriminator.summary()

c:\Users\ALI\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 16)   448         ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 16, 16, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 dropout (Dropout)              (None, 16, 16, 16)   0           ['leaky_re_lu[0][0]']            
                                                                                              

c:\Users\ALI\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
# create the generator
generator = define_generator(latent_dim)
generator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 110)          0           ['input_3[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 384)          42624       ['concatenate[0][0]']      

In [9]:
# make ac_gan model
gan_model = define_gan(generator, discriminator)
gan_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 110)          0           ['input_3[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 384)          42624       ['concatenate[0][0]']      

In [14]:
# make history
history = {'dr1':[], 'dr2':[], 'df1':[], 'df2':[], 'g1':[], 'g2':[]}
# load image data
dataset = load_real_samples()
epochs = 500

(60000, 32, 32, 3) (60000, 10)


In [11]:
# train model
history = train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=epochs, history=history)

Output hidden; open in https://colab.research.google.com to view.